# Import

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import pylab 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
sns.set()
%matplotlib inline

/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
train = pd.read_csv('../data/train.csv')
weather = pd.read_csv('../data/weather.csv')
key = pd.read_csv('../data/key.csv')
test = pd.read_csv('../data/test.csv')
sample = pd.read_csv('../data/sampleSubmission.csv')

# 함수

In [3]:
def by_store(df, nbr) : # sort by store number and re-index
    df_new = df[df['store_nbr'] == nbr]
    df_new.reset_index(drop = True, inplace = True)
    return df_new

In [4]:
def M_transform(df, column): #moving average (recursive)
    tmp_sorted = df.sort_values(by=['station_nbr','date']).reset_index(drop=True)
    weather_new = pd.DataFrame(columns=['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
           'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
           'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
           'avgspeed'])
    window = 2
    for j in range(1, 21):
        tmp_station = []
        tmp_station = tmp_sorted[tmp_sorted['station_nbr'] == j].reset_index(drop=True)
        if j == 5:
            weather_new = pd.concat([weather_new, tmp_station])
        else:
            for i in range(len(tmp_station)):
                if tmp_station[column].at[i] == 'M':
                    tmp = 0.0
                    if i == 0:
                        result = 0.0
                    else:
                        for x in range(1, window + 1):
                            tmp += float(tmp_station[column].at[i - x])
                            result = float(round(tmp / window, 3))
                            tmp_station[column].set_value(i, result)
            weather_new = pd.concat([weather_new, tmp_station]).reset_index(drop=True)
    return weather_new

In [5]:
def TM_transform(series, T_replace, M_replace):  # Temporary solution
    """
    데이터내의 T, M을 원하는 값으로 바꿔주는 함수
    TM_transform(series, T_replace)
    """
    series = series.astype(str).map(lambda s: s.strip())
    series[series == 'T'] = T_replace
    series[series == 'M'] = M_replace
    return series.astype('float')

In [6]:
def T_transform(series, T_replace): 
    """
    데이터내의 T, M을 원하는 값으로 바꿔주는 함수
    TM_transform(series, T_replace)
    """
    series = series.astype(str).map(lambda s: s.strip())
    series[series == 'T'] = T_replace
    series[series == 'M'] = 'M'
    return series

In [7]:
def get_item_nbr(df) : # 모든 units이 0이 아닌 item_nbr을 구하는 함수, list형태로 return
    tmp = df.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
    tmp = tmp.loc[:, (tmp != 0).any(axis = 0)]
    tmp.loc['2012-12-25'] = 0 # 2012-12-25가 빠져있음 train data에서.. 그래서 log_units = 0으로 넣어줌.
    
    tmp.reset_index(inplace = True)
    tmp.sort_values(by = 'date', inplace = True)
    tmp.drop(['date'], axis = 1, inplace = True)
    
    result = list(tmp.columns)
    
    return result

In [8]:
def match_dateformat(df, year):
    """
    영문 월을 숫자 월로 바꾸어주고 나중에 사용하기 쉽도록 datetime.date 형태로 바꾸어주는 함수
    """
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    for i in range(len(df)):
        dates = df.loc[i][0]
        dates = dates.split(" ")
        for j in range(len(months)):
            if dates[0] == months[j]:
                dates[0] = str(j + 1)
                dates_df = ["{} {} {}".format(year, dates[0], dates[1])]
                dates_df = pd.to_datetime(dates_df)
                df.loc[i][0] = dates_df.date[0]
    return df

In [9]:
def merge_holiday(holiday_df1, holiday_df2, holiday_df3):
    """
    각 연도별 공휴일 리스트 합치기
    """
    frame = [holiday_df1, holiday_df2, holiday_df3]
    holiday = pd.concat(frame).reset_index(drop=True)
    return holiday

def find_holiday(file, year):
    """
    수요에 영향을 미치는 주요 공휴일을 찾아내는 함수
    """
    holidays = ["New Year's Day", "Martin Luther King Jr. Day", "Valentine's Day",  "President's Day", "Easter Sunday", 
                      "Mother's Day", "Memorial Day", "Father's Day", "Independence Day", "Labor Day", "Columbus Day",
                      "Halloween", "Veterans Day", "Thanksgiving Day", "Black Friday", "Christmas Eve", "Christmas Day", "New Year's Eve"]
    
    holi = pd.read_excel(file, year, header=None)
    holi = match_dateformat(holi, year)
    holiday = pd.DataFrame(columns=[0,1,2,3,4])
    for _ in holidays:
        for i in range(len(holi[2])):
            if _ == holi[2][i]:
                holiday = holiday.append(holi.loc[i])
    return holiday

def cs_preprocessing(codesum):
    codesum_temp = []
    for _ in codesum:
        _ = _.replace('+', '')
        _ = _.replace('-', '')
        if len(_) > 2:
            _1 = _[:2]
            codesum_temp.append(_1)
            _2 = _[2:]
            codesum_temp.append(_2)
        else:
            codesum_temp.append(_)
    codesum = codesum_temp
    return codesum

def weather_flagger(weather):
    codesum_ls = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC']
    weather['date'] = pd.to_datetime(weather['date']) #weather는 글로벌변수
    for i in range(len(weather['codesum'])):
        codesum = weather['codesum'][i].split(" ")
        codesum = cs_preprocessing(codesum)
        for _ in codesum:
            flag = any(code in _ for code in codesum_ls)
            if flag == True:
                weather.set_value(i, '{}_flag'.format(_), 1)
            else:
                weather.set_value(i, 'normal_flag', 1)
    weather['snowfall'] = TM_transform(weather['snowfall'], 0.02, 0.0)
    weather['preciptotal'] = TM_transform(weather['preciptotal'], 0.02, 0.0)
    weather['snow_event'] = np.where(np.where(weather['SN_flag'] == 1, 1, 0) + np.where(weather['snowfall'] > 2, 1, 0) == 2, 1, 0)
    weather['rain_event'] = np.where(np.where(weather['RA_flag'] == 1, 1, 0) + np.where(weather['preciptotal'] > 1, 1, 0) == 2, 1, 0)
    weather['event'] = weather['snow_event'] + weather['rain_event']
    weather['event'] = np.where(weather['event'] >= 1, 1, 0)
    return weather

def preprocessing(df, holiday, weather):
    """
    train데이터를 가공하는 함수
    """
    df['log_units'] = np.log(df['units'] + 1) # logged units
    df['date'] = pd.to_datetime(df['date'])
    df['weekday'] = df.date.dt.weekday  # 월요일이 0 일요일이 6
    df['weekend'] = df.date.dt.weekday.isin([5, 6])  # 5: 토요일, 6: 일요일

    df['holiday'] = df.date.isin(holiday[0])
    df['weekday_holiday'] = df.holiday & (df.weekend == False)
    df['weekend_holiday'] = df.holiday & df.weekend
    
    df = pd.merge(df, key, on='store_nbr') #key는 글로벌 변수
    df = pd.merge(df, weather[['date', 'station_nbr', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool',
                               'sunrise', 'sunset', 'codesum', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed',
                               'resultspeed', 'resultdir', 'avgspeed' ,'event']], on=['date', 'station_nbr'])
    return df

def test_preprocessing(df, holiday, weather):
    weather = weather_flagger(weather)
    df['date'] = pd.to_datetime(df['date'])
    df['weekday'] = df.date.dt.weekday  # 월요일이 0 일요일이 6
    df['weekend'] = df.date.dt.weekday.isin([5, 6])  # 5: 토요일, 6: 일요일

    df['holiday'] = df.date.isin(holiday[0])
    df['weekday_holiday'] = df.holiday & (df.weekend == False)
    df['weekend_holiday'] = df.holiday & df.weekend
    
    df = pd.merge(df, key, on='store_nbr') #key는 글로벌 변수
    df = pd.merge(df, weather[['date', 'station_nbr', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool',
                               'sunrise', 'sunset', 'codesum', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed',
                               'resultspeed', 'resultdir', 'avgspeed' ,'event']], on=['date', 'station_nbr'])
    return df

In [10]:
train_df = train[train['date'] <= '2013-03-31'].reset_index(drop=True)
weather_df = weather[weather['date'] <= '2013-03-31'].reset_index(drop = True)

In [11]:
holiday12 = find_holiday('../data/holiday.xlsx', '2012')
holiday13 = find_holiday('../data/holiday.xlsx', '2013')
holiday14 = find_holiday('../data/holiday.xlsx', '2014')
holiday = merge_holiday(holiday12, holiday13, holiday14)
weather_df = weather_flagger(weather_df)
processed_train = preprocessing(train, holiday, weather_df)
processed_test = test_preprocessing(test, holiday, weather)

/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:50: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:52: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [12]:
processed_train = preprocessing(train_df, holiday, weather_df)
processed_train['date'] = processed_train['date'].apply(lambda x:x.date().strftime('%Y-%m-%d'))

In [13]:
contrast_weekday = np.eye(7)
result_test = [] #outlier 제거 전
result_test2 = [] #outlier 제거 후

In [14]:
processed_train.drop(['weekend', 'weekday_holiday', 'weekend_holiday',
       'station_nbr', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb',
       'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultspeed',
       'resultdir', 'avgspeed'], axis = 1, inplace = True)

In [15]:
processed_test.drop(['weekend', 'weekday_holiday', 'weekend_holiday', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'codesum', 'snowfall', 'preciptotal', 'stnpressure',
       'sealevel', 'resultspeed', 'resultspeed', 'resultdir', 'avgspeed'], axis = 1, inplace = True)

In [16]:
df_44 = processed_train[processed_train['store_nbr'] == 44].reset_index(drop = True)

model44 = sm.OLS.from_formula('log_units ~ C(item_nbr):C(weekday, contrast_weekday) + 0', data = df_44)
result_model44 = model44.fit()
print(result_model44.summary())

test_44 = processed_test[processed_test['store_nbr'] == 44]

test_44['log_units'] = result_model44.predict(test_44)
test_44['units'] = np.exp(test_44['log_units']) - 1
result_test.append(test_44)

                            OLS Regression Results                            
Dep. Variable:              log_units   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                     1055.
Date:                Wed, 14 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:07:00   Log-Likelihood:                 33270.
No. Observations:               50505   AIC:                        -6.499e+04
Df Residuals:                   49728   BIC:                        -5.812e+04
Df Model:                         776                                         
Covariance Type:            nonrobust                                         
                                                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [17]:
item_nbr_list_44 = get_item_nbr(df_44) #44번 store에서 팔린 item_nbr만 list로 저장

influence44 = result_model44.get_influence()

cooks_d2_44, pvals44 = influence44.cooks_distance # fox outlier recommendation으로 outlier 판별
fox_cr44 = 4 / (len(df_44) - 778)
idx44 = np.where(cooks_d2_44 > fox_cr44)[0]

print(len(idx44)) #outlier 갯수

for num in idx44 : # outlier들을 각 item_nbr별 평균치로 대체
    for item_nbr in item_nbr_list_44 :
        if item_nbr == df_44.loc[num].item_nbr :
            mean = df_44[df_44['item_nbr'] == item_nbr]['log_units'].mean()
            df_44.set_value(num, 'log_units', mean)
            df_44.set_value(num, 'units', np.exp(mean) - 1)

model44_new = sm.OLS.from_formula('log_units ~ C(item_nbr):C(weekday, contrast_weekday) + 0', data = df_44)
result_model44_new = model44_new.fit()
print(result_model44_new.summary())

test_44_new = processed_test[processed_test['store_nbr'] == 44]

test_44_new['log_units'] = result_model44_new.predict(test_44_new)
test_44_new['units'] = np.exp(test_44_new['log_units']) - 1
result_test2.append(test_44_new)

tmp_sub = pd.concat(result_test2)
tmp_sub.to_csv('test4.csv', index = False)

1642


/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


                            OLS Regression Results                            
Dep. Variable:              log_units   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 4.849e+04
Date:                Wed, 14 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:07:49   Log-Likelihood:             1.2996e+05
No. Observations:               50505   AIC:                        -2.584e+05
Df Residuals:                   49728   BIC:                        -2.515e+05
Df Model:                         776                                         
Covariance Type:            nonrobust                                         
                                                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
df_45 = processed_train[processed_train['store_nbr'] == 45].reset_index(drop = True)

model45 = sm.OLS.from_formula('log_units ~ C(item_nbr):C(weekday, contrast_weekday) + 0', data = df_45)
result_model45 = model45.fit()
print(result_model45.summary())

test_45 = processed_test[processed_test['store_nbr'] == 45]

test_45['log_units'] = result_model45.predict(test_45)
test_45['units'] = np.exp(test_45['log_units']) - 1
result_test.append(test_45)

                            OLS Regression Results                            
Dep. Variable:              log_units   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     352.4
Date:                Wed, 14 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:08:31   Log-Likelihood:                 35137.
No. Observations:               50505   AIC:                        -6.872e+04
Df Residuals:                   49728   BIC:                        -6.186e+04
Df Model:                         776                                         
Covariance Type:            nonrobust                                         
                                                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [19]:
item_nbr_list_45 = get_item_nbr(df_45) #45번 store에서 팔린 item_nbr만 list로 저장

influence45 = result_model45.get_influence()

cooks_d2_45, pvals45 = influence45.cooks_distance # fox outlier recommendation으로 outlier 판별
fox_cr45 = 4 / (len(df_45) - 778)
idx45 = np.where(cooks_d2_45 > fox_cr45)[0]

print(len(idx45)) #outlier 갯수

for num in idx45 : # outlier들을 각 item_nbr별 평균치로 대체
    for item_nbr in item_nbr_list_45 :
        if item_nbr == df_45.loc[num].item_nbr :
            mean = df_45[df_45['item_nbr'] == item_nbr]['log_units'].mean()
            df_45.set_value(num, 'log_units', mean)
            df_45.set_value(num, 'units', np.exp(mean) - 1)

model45_new = sm.OLS.from_formula('log_units ~ C(item_nbr):C(weekday, contrast_weekday) + 0', data = df_45)
result_model45_new = model45_new.fit()
print(result_model45_new.summary())

test_45_new = processed_test[processed_test['store_nbr'] == 45]

test_45_new['log_units'] = result_model45_new.predict(test_45_new)
test_45_new['units'] = np.exp(test_45_new['log_units']) - 1
result_test2.append(test_45_new)

tmp_sub = pd.concat(result_test2)
tmp_sub.to_csv('test4.csv', index = False)

856


/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


                            OLS Regression Results                            
Dep. Variable:              log_units   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 3.211e+04
Date:                Wed, 14 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:09:18   Log-Likelihood:             1.4856e+05
No. Observations:               50505   AIC:                        -2.956e+05
Df Residuals:                   49728   BIC:                        -2.887e+05
Df Model:                         776                                         
Covariance Type:            nonrobust                                         
                                                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yjoung90/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Make Submission

In [31]:
aaa = pd.read_csv('test2.csv')
bbb = pd.read_csv('test3.csv')
ccc = pd.read_csv('test4.csv')

In [32]:
sub = pd.concat([aaa, bbb, ccc])

In [34]:
len(sub), len(test)

(526917, 526917)

In [36]:
sub.drop(['weekday', 'holiday', 'event'], axis = 1, inplace = True)

In [37]:
sub.sort_values(by = ['date', 'store_nbr', 'item_nbr'], inplace = True)
sub.reset_index(drop = True, inplace = True)

In [38]:
units = sub['units']

In [39]:
sub['store_nbr'] = sub['store_nbr'].astype('str')
sub['item_nbr'] = sub['item_nbr'].astype('str')
sub['date'] = sub['date'].astype('str')

In [40]:
sub['id'] = sub['store_nbr'] + '_' + sub['item_nbr'] + '_' + sub['date']

In [41]:
sub.drop(['date', 'store_nbr', 'item_nbr', 'log_units', 'units'], axis = 1, inplace = True)

In [42]:
sub['units'] = units

In [43]:
sub.tail()

,id,units
526912,45_107_2014-10-26,0.0
526913,45_108_2014-10-26,0.0
526914,45_109_2014-10-26,0.0
526915,45_110_2014-10-26,0.0
526916,45_111_2014-10-26,0.0


In [44]:
sub.to_csv('sub_finally.csv', index = False)